<a href="https://colab.research.google.com/github/afvallejo/snPATHOSeq/blob/main/FRP_validation_using_LnCaP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**snPATHO-seq: unlocking the FFPE archives for single nucleus RNA profiling**



Andres F Vallejo<sup>1,2</sup>, Kate Harvey<sup>3</sup>, Taopeng Wang<sup>3,4</sup>, Kellie Wise<sup>1,2</sup>, Lisa M. Butler<sup>2,5</sup>, Jose Polo<sup>1,2</sup>, Jasmine Plummer<sup>6,7</sup>, Alex Swarbrick<sup>3,4,&</sup>, Luciano G Martelotto<sup>1,2,&,#</sup>


1.	ACE – Adelaide Centre for Epigenetics, University of Adelaide, South Australia, Australia. 
2.	SAiGENCI – South Australian immunoGENomics Cancer Institute, University of Adelaide, South Australia, Australia. 
3.	Cancer Ecosystems Program, Garvan Institute of Medical Research, Darlinghurst, NSW 2010, Australia
4.	St Vincent’s Clinical School, Faculty of Medicine and Health, UNSW Sydney, Kensington, NSW 2052, Australia.
5.	Solid Tumour Program, South Australian Health and Medical Research Institute, Adelaide, South Australia, Australia
6.	Department of Biomedical Sciences, Center for Bioinformatics and Functional Genomics, Cedars-Sinai Medical Center, Los Angeles, CA, 90048, USA
7.	Applied Genomics, Computation and Translational Core, Cedars Sinai Medical Center, Los Angeles, CA, 90048, USA

& Joint senior authors
## Correspondence to:
A/Prof. Luciano Martelotto, luciano.martelotto@adelaide.edu.au

#Colab

In [ ]:
# restart after installing

#for python 3.7
!pip install scanpy leidenalg pandas==1.1.5 umap-learn==0.5.3 plotnine==0.8.0 anndata==0.7.8 panel


In [1]:
!pip install scanpy leidenalg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 27.8 MB/s 
     |████████████████████████████████| 1.3 MB 44.5 MB/s 
     |████████████████████████████████| 88 kB 5.1 MB/s 
     |████████████████████████████████| 96 kB 2.6 MB/s 
     |████████████████████████████████| 11.2 MB 39.6 MB/s 
     |████████████████████████████████| 956 kB 63.5 MB/s 
     |████████████████████████████████| 1.1 MB 8.6 MB/s 
     |████████████████████████████████| 3.1 MB 37.4 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=738a28e4b508c1b78dd708520e50085a2993956150e0483baaadb9618c73be09
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=593a25cf92cf554c15f0e0c0a

In [1]:
import scanpy as sc
import numpy as np
import re
import os
import pandas as pd
import seaborn as sb
import matplotlib
import scipy.sparse as sp
from scipy.sparse import issparse

import matplotlib as mpl
from scipy import stats as scistats
import matplotlib.pyplot as pl
import scanpy.external as sce
from matplotlib import colors
from pathlib import Path


sc._settings.ScanpyConfig.n_jobs=4





# Downloading data

## Fresh LnCaP

In [4]:
!wget -v -O Fresh.h5ad https://universityofadelaide.box.com/v/LnCaP-Fresh-filtered

--2022-08-24 07:30:52--  https://universityofadelaide.box.com/v/LnCaP-Fresh-filtered
Resolving universityofadelaide.box.com (universityofadelaide.box.com)... 185.235.236.197
Connecting to universityofadelaide.box.com (universityofadelaide.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://universityofadelaide.app.box.com/v/LnCaP-Fresh-filtered [following]
--2022-08-24 07:30:53--  https://universityofadelaide.app.box.com/v/LnCaP-Fresh-filtered
Resolving universityofadelaide.app.box.com (universityofadelaide.app.box.com)... 74.112.186.144
Connecting to universityofadelaide.app.box.com (universityofadelaide.app.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /login?redirect_url=https%3A%2F%2Funiversityofadelaide.app.box.com%2Fv%2FLnCaP-Fresh-filtered [following]
--2022-08-24 07:30:53--  https://universityofadelaide.app.box.com/login?redirect_url=https%3A%2F%2Funi

In [19]:
!wget https://universityofadelaide.app.box.com/file/1000527797140?s=2ywzhgpn96o6aceezxbrxzoji0kk6emv

--2022-08-24 07:48:27--  https://universityofadelaide.app.box.com/file/1000527797140?s=2ywzhgpn96o6aceezxbrxzoji0kk6emv
Resolving universityofadelaide.app.box.com (universityofadelaide.app.box.com)... 74.112.186.144
Connecting to universityofadelaide.app.box.com (universityofadelaide.app.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://universityofadelaide.box.com/s/2ywzhgpn96o6aceezxbrxzoji0kk6emv [following]
--2022-08-24 07:48:27--  https://universityofadelaide.box.com/s/2ywzhgpn96o6aceezxbrxzoji0kk6emv
Resolving universityofadelaide.box.com (universityofadelaide.box.com)... 107.152.29.197
Connecting to universityofadelaide.box.com (universityofadelaide.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://universityofadelaide.app.box.com/s/2ywzhgpn96o6aceezxbrxzoji0kk6emv [following]
--2022-08-24 07:48:27--  https://universityofadelaide.app.box.com/s/2yw

In [ ]:
wget -v -O myfile.tgz -L https://ibm.box.com/shared/static/xxxxx.tgz

In [7]:
!curl -L  https://universityofadelaide.box.com/s/2ywzhgpn96o6aceezxbrxzoji0kk6emv --output Fresh2.h5ad

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 13276    0 13276    0     0  12802      0 --:--:--  0:00:01 --:--:--     0


![image.png]()